<a href="https://colab.research.google.com/github/bodamohannaik/Udemy_DL/blob/master/rbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries  

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch

# Download Dataset

In [2]:
!curl -O  http://files.grouplens.org/datasets/movielens/ml-100k.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4808k  100 4808k    0     0  22.5M      0 --:--:-- --:--:-- --:--:-- 22.5M
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


In [3]:
!unzip -q ml-100k.zip
!ls

replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


In [4]:
!curl -O  http://files.grouplens.org/datasets/movielens/ml-1m.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  22.4M      0 --:--:-- --:--:-- --:--:-- 22.4M
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


In [5]:
!unzip -q ml-1m.zip
!ls

replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


# Load Dataset

In [6]:
movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, engine='python', encoding='latin-1')
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
users = pd.read_csv('ml-1m/users.dat', delimiter='::', header=None, engine='python', encoding='latin-1')
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [8]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, engine='python', encoding='latin-1')
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


# prepare Data

In [9]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None, engine='python', encoding='latin-1')
training_set

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [10]:
testing_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None, engine='python', encoding='latin-1')
testing_set

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
...,...,...,...,...
19995,458,648,4,886395899
19996,458,1101,4,886397931
19997,459,934,3,879563639
19998,460,10,3,882912371


In [11]:
# all user ids
user_ids = list(set(training_set.iloc[:, 0].unique().tolist()+testing_set.iloc[:, 0].unique().tolist()))
user_ids.sort()
len(user_ids)

943

In [12]:
movie_ids = list(set(training_set.iloc[:, 1].unique().tolist()+testing_set.iloc[:, 1].unique().tolist()))
movie_ids.sort()
len(movie_ids)

1682

In [13]:
def convert(data):
  df = pd.DataFrame(index=user_ids, columns = movie_ids)
  for ind,record in data.iterrows():
    df.loc[record[0], record[1]] = record[2]
  df.fillna(value =0, inplace=True)
  return df.values.tolist()
training_set_converted = convert(data = training_set)
testing_set_converted = convert(data = testing_set)

In [14]:
train_set = torch.FloatTensor(training_set_converted)
test_set = torch.FloatTensor(testing_set_converted)

In [15]:
train_set.shape, test_set.shape

(torch.Size([943, 1682]), torch.Size([943, 1682]))

In [16]:
train_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [17]:
# convert to binary ratings
train_set[train_set == 0] = -1
train_set[train_set == 1] = 0
train_set[train_set == 2] = 0
train_set[train_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [18]:
train_set

tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

# RBM

In [31]:
# Neural Network
class RBM:
  def __init__(self, nv, nh):
    """
    :param nv: number of visible nodes
    :param nh: number of hidden nodes
    """
    # initialize weights
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  def sample_h(self, x):
    """"
    :param x: visible nodes
    """
    wx = torch.mm(x, self.W.t())
    wz = wx+self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(wz)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  def sample_v(self, y):
    """"
    :param y: hidden nodes
    """
    wy = torch.mm(y, self.W)
    wz = wy+self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(wz)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  def train(self,v0, vk, ph0, phk):
    """
    :param v0: initial visible nodes
    :param vk: visible nodes after k sampling steps
    :param ph0: initial hidden nodes probabilities
    :param phk: hidden nodes probabilities after k sampling steps
    """
    self.W = self.W+ (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b = self.b+ torch.sum(v0-vk, 0)
    self.a = self.a+ torch.sum(ph0-phk, 0)

  

# Train RBM

In [32]:
nv = train_set.shape[1]
nh = 100
batch_size =100
rbm = RBM(nv = nv, nh = nh)

In [33]:
# train
np_epochs = 10
for epoch in range(1, np_epochs+1):
  train_loss = 0
  s = 0.0
  for user_id in range(0, len(user_ids), batch_size):
    v0 = train_set[user_id:user_id+batch_size]
    ph0, _ = rbm.sample_h(v0)
    # initialize 
    vk = v0
    for k in range(10):
      _, hk = rbm.sample_h(vk)
      _, vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    # probabilities 
    phk, _ = rbm.sample_h(vk)
    # update weights
    rbm.train(v0 = v0, vk = vk, ph0 = ph0, phk = phk)
    # update loss
    train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
    s += 1
  print(f"epoch:{epoch}, loss:{train_loss/s}")

epoch:1, loss:0.3456597924232483
epoch:2, loss:0.25000321865081787
epoch:3, loss:0.24901323020458221
epoch:4, loss:0.2473345696926117
epoch:5, loss:0.24797263741493225
epoch:6, loss:0.24573621153831482
epoch:7, loss:0.24645563960075378
epoch:8, loss:0.24440404772758484
epoch:9, loss:0.24828307330608368
epoch:10, loss:0.2461199313402176


# Test RBM

In [35]:
test_loss = 0
s = 0.0
for user_id in range(0, len(user_ids), batch_size):
  v_in = train_set[user_id:user_id+batch_size]
  v_test = test_set[user_id:user_id+batch_size]
  if len(v_test[v_test>=0]) > 0:
    _, h = rbm.sample_h(v_in)
    _, v = rbm.sample_v(h)
    # update loss
    test_loss += torch.mean(torch.abs(v_test[v_test>=0]-v[v_test>=0]))
    s += 1
print(f"train loss:{test_loss/s}")

train loss:0.22980614006519318
